In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 11 07:13:23 2025

@author: lizamclatchy
"""

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

#Using a regression model to adequately predict wind data, not forecasting, learning and predicting missing
combined_df = pd.read_csv("/Users/lizamclatchy/ASCDP/Data Cleaning/Cleaned Model Input Data/train_afono_windspeed.csv")
selected_columns = ['TIMESTAMP', 'WS_mph_S_WVT'] + [col for col in combined_df.columns if col not in ['TIMESTAMP', 'WS_mph_S_WVT']]
rh_data = combined_df[selected_columns].copy()
rh_data = rh_data.dropna(subset=['WS_mph_S_WVT'])  # Keep only rows where RH_Aasu is not NaN

def feature_engineering(df):
    df = df.copy()
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    target_column = 'WS_mph_S_WVT'
    feature_cols = [col for col in df.columns if col not in ['TIMESTAMP', target_column,'Elevation_target','synoptic_elevation_1','synoptic_elevation_0']]    
    for col in feature_cols:
        df[f'{col}_lag1'] = df[col].shift(2)
        df[f'{col}_lag3'] = df[col].shift(5)
        df[f'{col}_lag6'] = df[col].shift(7)
        df[f'{col}_rolling2'] = df[col].rolling(window=2).mean()
        df[f'{col}_rolling4'] = df[col].rolling(window=4).mean()
        df[f'{col}_rolling6'] = df[col].rolling(window=6).mean()
    
    # Add more granular time features
    df['hour_of_day'] = pd.to_datetime(df['TIMESTAMP']).dt.hour
    df['is_daytime'] = ((df['hour_of_day'] >= 6) & (df['hour_of_day'] <= 18)).astype(int)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)

    # Day of Week (0=Monday, 6=Sunday)
    df['day_of_week'] = df['TIMESTAMP'].dt.dayofweek
    # Season (simplified meteorological)
    df['month'] = df['TIMESTAMP'].dt.month
    def get_season(month):
        if month in [12, 1, 2]:
            return "winter"
        elif month in [3, 4, 5]:
            return "spring"
        elif month in [6, 7, 8]:
            return "summer"
        else:
            return "fall"
    
    df['season'] = df['month'].apply(get_season)
    # Enforce season as categorical with all 4 categories
    df['season'] = pd.Categorical(df['season'], categories=["winter", "spring", "summer", "fall"])
    # One-hot encode with fixed categories
    season_dummies = pd.get_dummies(df['season'], prefix='season')
    season_dummies = season_dummies.astype(int)
    df = pd.concat([df, season_dummies], axis=1)
    df.drop(columns=['season'], inplace=True)

    # Heat Index (approximation using air temp and RH)
    if 'AirTF_target' in df.columns and 'RH_target' in df.columns:
        df['heat_index_target'] = df['AirTF_target'] * df['RH_target'] / 100
        df['temp_trend_1h_target'] = df['AirTF_target'].rolling(6).apply(lambda x: x.iloc[-1] - x.iloc[0], raw=False)
    for col in df.columns:
        if col != 'TIMESTAMP':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Now add interaction terms here (currently commented out)
    df['wind_per_rh'] = df['wind_speed_weighted_0_rolling6'] / (df['relative_humidity_weighted_0_rolling6'] + 1e-3)
    df['solar_per_temp'] = df['SolarMJ_target_rolling4'] / (df['air_temp_weighted_0_rolling6'] + 1e-3)
    
    interaction_pairs = [
    ('RH_target', 'air_temp_weighted_1_rolling2'),
    ('RH_target_rolling2', 'air_temp_weighted_0_rolling4'),
    ('relative_humidity_weighted_1_rolling2', 'air_temp_weighted_0_rolling2'),
    ('SolarMJ_target_lag1', 'RH_target_rolling2'),
    ('SolarMJ_target_rolling4', 'air_temp_weighted_1_rolling2'),
    ('SolarMJ_target_rolling4', 'relative_humidity_weighted_1_rolling2'),
    ('wind_speed_weighted_1_rolling6', 'RH_target'),
    ('wind_speed_weighted_1_rolling6', 'SolarMJ_target_rolling4'),
    ('season_summer', 'RH_target_rolling2'),
    ('season_summer', 'SolarMJ_target_lag1'),
    ('season_summer', 'PTemp_target'),
]
    
    for col1, col2 in interaction_pairs:
        if col1 in df.columns and col2 in df.columns:
            df[f'{col1}__X__{col2}'] = df[col1] * df[col2]
    
    return df
    

def cross_validate_model(model, X, y, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    train_scores, val_scores = [], []

    for train_idx, val_idx in tscv.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_tr, y_tr)
        train_pred = model.predict(X_tr)
        val_pred = model.predict(X_val)

        train_mae = mean_absolute_error(y_tr, train_pred)
        val_mae = mean_absolute_error(y_val, val_pred)

        train_scores.append(train_mae)
        val_scores.append(val_mae)

        print(f"Fold {len(train_scores)} - Train MAE: {train_mae:.4f}, Val MAE: {val_mae:.4f}")

    return train_scores, val_scores


def xgboost_regression(train_df, target_column, n_splits=5):
    # --- 1. Prepare Data ---
    train_df = train_df.sort_values(by="TIMESTAMP")
    train_data, test_data = train_test_split(train_df, test_size=0.2, shuffle=False)
    train_data = feature_engineering(train_data).dropna()
    test_data = feature_engineering(test_data).dropna()

    X_train = train_data.drop(columns=[target_column, 'TIMESTAMP'])
    y_train = train_data[target_column]
    X_test = test_data.drop(columns=[target_column, 'TIMESTAMP'])
    y_test = test_data[target_column]

    # --- 2. Visualize Train-Test Split ---
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.plot(train_data.index, y_train, label='Training Set')
    ax.plot(test_data.index, y_test, label='Test Set', linestyle="dashed")
    ax.set_title(f'Data Train/Test Split for {target_column}')
    ax.legend()
    plt.show()

    # --- 3. Define Objective Function for Optuna with CV ---
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 1),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
            'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
            'objective': 'reg:squarederror',
            'random_state': 42
        }

        model = XGBRegressor(**params)
        tscv = TimeSeriesSplit(n_splits=n_splits)
        mae_scores = []
    
        for train_index, val_index in tscv.split(X_train):
            X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
            y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
            model.fit(X_tr, y_tr)
            preds = model.predict(X_val)
            mae_scores.append(mean_absolute_error(y_val, preds))
    
        return np.mean(mae_scores)

    # --- 4. Run Optuna ---
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)
    best_params = study.best_params
    print("Best Params:", best_params)

    # --- 5. Cross-Validate Best Model and Default Model ---
    best_model = XGBRegressor(**best_params)
    default_model = XGBRegressor(objective='reg:squarederror', random_state=42)

    train_mae_best, val_mae_best = cross_validate_model(best_model, X_train, y_train, n_splits=n_splits)
    train_mae_default, val_mae_default = cross_validate_model(default_model, X_train, y_train, n_splits=n_splits)

    # --- 6. Plot CV Results ---
    folds = range(1, len(train_mae_best) + 1)
    plt.figure(figsize=(12, 6))
    plt.plot(folds, train_mae_default, 'o-', label='Default Train MAE')
    plt.plot(folds, val_mae_default, 'o--', label='Default Val MAE')
    plt.plot(folds, train_mae_best, 's-', label='Tuned Train MAE')
    plt.plot(folds, val_mae_best, 's--', label='Tuned Val MAE')
    plt.xlabel("Fold")
    plt.ylabel("MAE")
    plt.title("Train vs Validation MAE per Fold (Default vs Tuned)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # --- 7. Fit Final Model and Evaluate ---
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    from sklearn.linear_model import LinearRegression
    lr = LinearRegression().fit(X_train, y_train)
    print("MAE (Linear):", mean_absolute_error(y_test, lr.predict(X_test)))

    smape_error = smape(y_test, y_pred)
    print(f"SMAPE: {smape_error:.2f}%")
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MAE: {mae:.4f} mph")
    print(f"R²:    {r2:.4f}")

    # --- 8. Plot Final Prediction ---
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual', color='blue')
    plt.plot(y_test.index, y_pred, label='Predicted', linestyle='--', color='red')
    plt.title(f'Actual vs Predicted Wind Speed ({target_column})')
    plt.xlabel("Index")
    plt.ylabel("Wind Speed (log-transformed)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    from xgboost import plot_importance

    plt.figure(figsize=(10, 6))
    plot_importance(best_model, importance_type='gain', max_num_features=15, title='XGBoost Feature Importance')
    plt.tight_layout()
    plt.show()
    return best_model, y_test, y_pred

model, y_test, y_pred = xgboost_regression(rh_data, 'WS_mph_S_WVT')
#fe_df = feature_engineering(rh_data)
#fe_df.set_index("TIMESTAMP").to_csv("feature_engineered_data_with_index.csv")

plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test, label='Actual WS_mph_S_WVT_Afono', color='blue')
plt.plot(y_test.index, y_pred, label='Predicted RWS_mph_S_WVT_Afono', color='red', linestyle='--')
plt.xlabel('Index')
plt.ylabel('Speed (MPH)')
plt.title('Actual vs Predicted WS_mph_S_WVT_Afono')
plt.legend()
plt.tight_layout()
plt.show()


residuals = y_test - y_pred
plt.figure(figsize=(12, 4))
plt.plot(residuals, label='Residuals')
plt.axhline(0, color='black', linestyle='--')
plt.title("Prediction Residuals Over Time")
plt.legend()
plt.tight_layout()
plt.show()


